In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#import os

In [3]:
#os.chdir('drive/My Drive/ssd.pytorch')

In [4]:
import torch
from torch.autograd import Variable
import cv2
from data import BaseTransform, VOC_CLASSES as labelmap
from ssd import build_ssd
import imageio
import matplotlib.pyplot as plt
import time
import numpy as np
import sys
from sort import *
import random

%matplotlib inline

In [5]:
device = torch.device("cuda")
torch.set_default_tensor_type('torch.cuda.FloatTensor')

In [6]:
mot_tracker = Sort() 
net = build_ssd('test',300,2).cuda()
net.load_state_dict(torch.load('weights/ssd_person.pth'))

In [7]:
transform = BaseTransform(net.size, (104/256.0, 117/256.0, 123/256.0))

In [8]:
COLORS = np.random.rand(32,3)*256
FONT = cv2.FONT_HERSHEY_PLAIN
count = 0

In [9]:
def process_image(image):
    height, width = image.shape[:2]
    x = torch.from_numpy(transform(image)[0]).permute(2, 0, 1)
    x = Variable(x.unsqueeze(0))
    x = x.cuda()
    
    y = net(x)
    
    detections = y.data
    scale = torch.Tensor([width-width*.025, height-height*.05, width+width*.025, height+height*.05])
    
    allbox = []
    for i in range(detections.size(2)):
        if detections[0, 1, i, 0] >= .5:
            pt = (detections[0, 1, i, 1:] * scale).cpu().numpy()
            bbox = np.append(pt, (detections[0, 1, i, 0]).cpu().numpy())
            allbox.append(bbox)
    
    allbbox = np.array(allbox)
    track_bbs_ids = mot_tracker.update(allbbox)   

    for d in track_bbs_ids:
        t_size = cv2.getTextSize('person', cv2.FONT_HERSHEY_PLAIN, 1 , 1)[0]
        color = COLORS[int(d[4])%31]
        cv2.rectangle(image,(int(d[0]), int(d[1])),(int(d[2]),int(d[3])), color, 2)
        cv2.putText(image, (str(int(d[4]))), (int((d[0]+d[2])/2), int(d[1]) + int(t_size[1]) + 4), FONT, 1.5, color, 1, cv2.LINE_AA)
    
    return(image)

In [10]:
def process_video(video,ext):
    cap = cv2.VideoCapture('test_videos/'+video+'.'+ext)
    fps = int(cap.get(5))
    height, width = int(cap.get(3)), int(cap.get(4))
    writer = cv2.VideoWriter("output/"+video+'.mp4', cv2.VideoWriter_fourcc(*"mp4v"), fps,(height, width))
    i=0
    start = time.time()
    while(cap.isOpened()):
        ret, frame = cap.read()
        if ret==True:
            frame = process_image(frame)
            writer.write(frame)
            cv2.imshow("frame", frame)
            key = cv2.waitKey(1) & 0xFF
            if(key==ord('q')):
                break
            fps = i/(time.time()-start)
            sys.stdout.write('\rFPS = '+str(fps))
            i=i+1
        else:
            break
    cap.release()
    print("\n Done! and saved to {}".format('output/'+video+'.mp4'))
    writer.release()
    cv2.destroyAllWindows()

In [11]:
process_video('max','mp4')

FPS = 1.64939899611955656
 Done! and saved to output/max.mp4


In [12]:
#process_video('4p','avi')

In [13]:
#process_video('terrace','avi')

In [14]:
#process_video('Pedestrian','mp4')

In [15]:
#process_video('Town','avi')

In [16]:
#process_video('pedestrianwalk','mp4')

In [17]:
cv2.destroyAllWindows()